In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import glob
import os
import dask.dataframe as dd

### Concat using time_id

In [2]:
files = glob.glob(os.path.join("./individual_book_train", "*.csv"))

def concat_stocks(files, time_id = 5):
    
    # CREATE NEW DATAFRAME - with columns: stock_id and average wap for time_id
    stocks = pd.DataFrame({"stock_id":[], "wap":[], "seconds": []})
    
    for f in files[:]: 

        # LAZY READ TO OPTIMIZE READ TIME
        df = dd.read_csv(f)

        # FIND ONLY COLUMNS WITH time_id ONLY
        df = df[df['time_id'] == time_id]
        df = df.compute()
        
        # COMPUTE WAP COLUMN
        df["wap"] = (df["bid_price1"] * df["ask_size1"] + df["ask_price1"] * df["bid_size1"]) \
                    / (df["bid_size1"] + df["ask_size1"])
        
        d = pd.DataFrame({"stock_id": df["stock_id"], "wap": df["wap"], "seconds": df["seconds_in_bucket"]})
        
        ## FILLING ALL SECONDS WITH STATS FROM SECOND BEFORE (NO MISSING SECONDS)
        i = 0
        while i < 600: 
            if i == len(d) and i < 600 or d.loc[i].seconds > i:
                new = pd.DataFrame({'stock_id':[d.loc[i-1].stock_id], 'wap':[d.loc[i-1].wap], 'seconds':[i]})
                d = pd.concat([d, new]).sort_values('seconds')
                d = d.reset_index(drop=True)
        
            i += 1
    
        stocks = pd.concat([stocks, d]).astype({'stock_id':'int', 'seconds':'int'})
                
    return stocks

### Creating dataframe (may take some time)
- All stocks concatenated by time id

In [ ]:
s = concat_stocks(files, time_id = 5)

s

## Computing Beta

In [ ]:
def compute_beta(d):
    """
    Computes Beta coeff as Covariance(Stock,Market)/Variance(Market)
    Details found here: https://corporatefinanceinstitute.com/resources/data-science/beta-coefficient/
    """
    c = (((d['wap'] - d['wap_mean']) * (d['market_mean_seconds'] - d['market_mean'])) / len(d)).sum()
    #v = (((d['market_mean_seconds'] - d['market_mean']) ** 2) / len(d)).sum()
    d['beta'] = c/np.var(d['market_mean_seconds'])
    
    return d

In [ ]:
# Calculate average wap for stock
s['wap_mean'] = s.groupby(['stock_id'])['wap'].transform('mean')

# Calculate market mean for each second
s['market_mean_seconds'] = s.groupby(['seconds'])['wap'].transform('mean')

# Calculate overall market mean 
s['market_mean'] = s['wap'].mean()

# compute beta for each stock_id
s = s.groupby(['stock_id']).apply(compute_beta)

In [ ]:
s

In [ ]:
stock_betas = pd.DataFrame({'stock_id':s['stock_id'].unique(), 'beta':s['beta'].unique()})

stock_betas

### MeanShift (num clusters decided by data)

In [ ]:
from sklearn.cluster import MeanShift, estimate_bandwidth

sb = stock_betas
X = np.reshape(sb['beta'].values, (-1,1))
X

In [ ]:
ms = MeanShift(bandwidth=None, bin_seeding=True)
ms.fit(X)
labels = ms.labels_
cluster_centers = ms.cluster_centers_

labels_unique = np.unique(labels)
n_clusters_ = len(labels_unique)

print("number of estimated clusters : %d" % n_clusters_)
print(labels)

### K-means (num clusters will need be decided) - atm using Elbow

In [ ]:
from sklearn.cluster import KMeans

wcss = []

sb = stock_betas
X = np.reshape(sb['beta'].values, (-1,1))
X

for i in range(2, 17):
    kmeans = KMeans(n_clusters = i, init = "k-means++", max_iter = 300, n_init = 10, random_state = 0)
    kmeans.fit(X)
    wcss.append(kmeans.inertia_)
    
plt.plot(range(2, 17), wcss)
plt.title('The elbow method')
plt.xlabel('Number of clusters')
plt.ylabel('WCSS') #within cluster sum of squares
plt.show()

In [ ]:
km = KMeans(6)

clusts = km.fit_predict(X)
clusts

## Assigning Cluster IDs

In [ ]:
stock_betas['cluster_id'] = clusts
stock_betas

### Analysing clusters

In [ ]:
stock_betas.boxplot(column = 'beta', by='cluster_id')

In [ ]:
stock_betas.groupby(['cluster_id'])['beta'].median()

In [ ]:
stock_betas.groupby(['cluster_id'])['beta'].min()

In [ ]:
stock_betas.groupby(['cluster_id'])['beta'].max()

### Extra: Code found online for comparing cluster quality with selected K

In [ ]:
from sklearn.cluster import KMeans
from sklearn import metrics

X_segmentation = X

search_range = range(2, 18)
report = {}
for k in search_range:
    temp_dict = {}
    kmeans = KMeans(k)
    #inertia = kmeans.inertia_
    #temp_dict['Sum of squared error'] = inertia
    
    cluster = kmeans.fit_predict(X_segmentation)
    
    inertia = kmeans.inertia_
    temp_dict['Sum of squared error'] = inertia
    chs = metrics.calinski_harabasz_score(X_segmentation, cluster)
    ss = metrics.silhouette_score(X_segmentation, cluster)
    temp_dict['Calinski Harabasz Score'] = chs
    temp_dict['Silhouette Score'] = ss
    report[k] = temp_dict
    

report_df = pd.DataFrame(report).T
report_df.plot(figsize=(15, 10),
               xticks=search_range,
               grid=True,
               title=f'Selecting optimal "K"',
               subplots=True,
               marker='o',
               sharex=True)
plt.tight_layout()

In [ ]:
#!pip install yellowbrick
from yellowbrick.cluster.elbow import kelbow_visualizer

kelbow_visualizer(KMeans(random_state=1),
                  X_segmentation,
                  k=(2, 19),
                  timings=False)